# Hyperparameter Tuning of Random Forest Model
Workflow: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Here are the commands I used to tune the hyperparameters of the random forest model. I followed the workflow above and adjusted the commands to use a Random Forest Classifier model rather than a Random Forest Regression model. 

For more details on the analysis and input data createion, refer to the README file located on rustang/canopus: /home/wyatte/4DGB_ML_Random_Forest

In [ ]:
# Read in bed data
import pandas as pd
data = pd.read_csv("/Users/wyatteng/Desktop/4dgb/8.9.21_RandomForest/10.18.21_CTCF_RF_Input/CTCF_4000_R1_H3K27ac_RF_Input.bed",delimiter="\t")
data.head()

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
# Split dataset into TRAINING set and TEST set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False) # 70% training and 30% test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

## Plotting Feature Importance
You can plot the feature importance of a Random Forest model by selecting which features you would like to compare and identify them in the 'feature_names' list. After the model is trained, you can run the plotting cell to plot the feature importance.

In [ ]:
#Create a Gaussian Classifier
model=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train,y_train)

feature_names = ['Numreads', 'Covbases', 'Coverage', 'Meanbaseq', 'Sumdepth']
import pandas as pd
feature_imp = pd.Series(model.feature_importances_,index=feature_names).sort_values(ascending=False)
feature_imp

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

## Randomized Search (Hyperparameter Tuning)

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
# Max 500
n_estimators = [100,300,500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
# Max Depth upper limit = 30
# 3-10, 15, 20
max_depth = [3,4,5,6,7,8,9,10,15,20,25,30]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 20]
# Minimum number of samples required at each leaf node
# 1-16
min_samples_leaf = [1, 2, 4, 6, 8, 10, 12, 14, 16]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

Fitting the following code will take a LONG time. The larger the grid, the more combinations of RF runs will be ran.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
# The following command will output the best parameters found during the random grid search technique
rf_random.best_params_

Best parameters from random grid:

{'n_estimators': 300,
 'min_samples_split': 5,
 'min_samples_leaf': 16,
 'max_features': 'auto',
 'max_depth': 9,
 'bootstrap': True}
 
 The function below will print the model performance, precision, recall, and f1 score for the predicted results

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    print('Model Performance')
    print("Accuracy:",metrics.accuracy_score(test_labels, predictions))
    print(classification_report(test_labels, predictions))
    return predictions

base_model = RandomForestClassifier(n_estimators = 100)
base_model.fit(X_train, y_train)
random_predictions = evaluate(base_model, X_test, y_test)

In [ ]:
max_depth = [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, None]
md_results = dict()
accuracy_results = dict()
for item in max_depth:
    best_model = RandomForestClassifier(n_estimators=300,min_samples_split=5, min_samples_leaf=16, max_features='auto', max_depth=item, bootstrap=True)
    best_model.fit(X_train, y_train)
    predictions = best_model.predict(X_test)
    mse = metrics.mean_squared_error(y_test, predictions)
    ac = metrics.accuracy_score(y_test, predictions, normalize=True, sample_weight=None)
    md_results[item] = mse
    accuracy_results[item] = ac
    
print(md_results)
print(accuracy_results)

The code above fits the Random Forest model to the training dataset based on a varying max_depth. The model is then tested on the test dataset and the accuracy and mean squared error is stored in the respecting dictionaries. The plots below will visualize the results.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(md_results.keys(), md_results.values())
plt.title('MSE vs Max Depth')
plt.xlabel('Max Depth')
plt.ylabel('Mean Squared Error')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(accuracy_results.keys(), accuracy_results.values())
plt.title('Accuracy vs Max Depth')
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.show()

## Grid Search (Hyperparameter Tuning)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [3, 6, 9, 15, 20, 25, 30, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 300, 500]
}
# Create a based model

rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

The best results I found from grid_search technique was...

{'bootstrap': True,
 'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 300}

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    print('Model Performance')
    print("Accuracy:",metrics.accuracy_score(test_labels, predictions))
    print(classification_report(test_labels, predictions))
    return predictions

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

### Trying to Compare True vs Predicted
The code below was used to manipulate the data so the predictions and true data are in the same datatype.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_random = RandomForestClassifier(n_estimators=300,min_samples_split=5, min_samples_leaf=16, max_features='auto', max_depth=9, bootstrap=True)
best_model = rf_random
best_model.fit(X_train, y_train)
predictions = best_model.predict(X_test)

The 'true_val' data is the the true data that is converted to a numpy.ndarray in order to match the 'predictions' data type.

In [ ]:
X_axis=X_test.index.values
true_val=X_test.CTCF_Binding_Sites.values

I have yet to figure out a good way to compare the true and predicted data sets. The code below is a broad view of the two datasets, but it is not very informative since the X_axis is spread over a wide range

In [ ]:
from matplotlib import pyplot as plt
plt.plot(X_axis, real_array)
plt.plot(X_axis, predictions)
plt.title('True vs Predicted')
plt.xlabel('X index')
plt.ylabel('Predicted ATAC Peaks')
plt.show()